<a href="https://colab.research.google.com/github/laxmi1707/RAG_ApplicationForPDF/blob/main/Apple_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is to do a RAG application over some apple report . It provides domain specific review.
Problem statement
Develop a RAG application to help business analysts efficiently extract key insights from extensive reports, such as How Apple is Organized for Innovation, enabling faster and more informed decision-making.

In [1]:
#For GPU llma cpu python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 131.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 276.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 196.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 191.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.13 requires numpy<2,>=1, but you have numpy 2.3.1 which is incompatible.
langchain 0.1.1 requires numpy<2,>=1, but you have numpy 2.3.1 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 

In [2]:
!pip install huggingface_hub==0.23.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 q

  Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.1
    Uninstalling numpy-2.3.1:
      Successfully uninstalled numpy-2.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
peft 0.15.2 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.23.2 which is incompatible.
diffusers 0.34.0 requires huggingface-hub>=0.27.0, but you have huggingface-hub 0.23.2 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 wh

In [1]:
import json,os
import numpy as np
import tiktoken
import pandas as pd


#To do chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
#This is to load pdf
from langchain_community.document_loaders import PyMuPDFLoader
#For the embeddings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
#For the vector DB
from langchain_community.vectorstores import Chroma

#download the model from hf
from huggingface_hub import hf_hub_download
#this model is from hf
from llama_cpp import  Llama


#Data Prep for RAG

## Loading the data

In [2]:
from google.colab import drive

apple_pdf_path = "HBR_How_Apple_Is_Organized_For_Innovation-4.pdf"
path ="/content/drive/MyDrive/AIMLUseCaseProjects/DataSetRAG/"
drive.mount('/content/drive')


pdf_loader = PyMuPDFLoader(path+apple_pdf_path)
apple= pdf_loader.load()

Mounted at /content/drive


#Data Overview

##Checking the first 3 pages

In [4]:
for i in range(3):
  print(f"Page num:{i+1}", end="\n")
  print(apple[i].page_content, end="\n")

Page num:1
REPRINT R2006F
PUBLISHED IN HBR
NOVEMBER–DECEMBER 2020
ARTICLE
ORGANIZATIONAL CULTURE
How Apple Is 
Organized  
for Innovation
It’s about experts leading experts. 
by Joel M. Podolny and Morten T. Hansen
This article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.

Page num:2
2
Harvard Business Review
November–December 2020
This article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.

Page num:3
PHOTOGRAPHER MIKAEL JANSSON
How Apple Is  
Organized  
for Innovation
It’s about experts  
leading experts.
ORGANIZATIONAL 
CULTURE
Joel M. 
Podolny
Dean, Apple 
University
Morten T. 
Hansen
Faculty, Apple 
University
AUTHORS
FOR ARTICLE REPRINTS CALL 800-988-0886 OR 617-783-7500, OR VISIT HBR.ORG
Harvard Business Review
November–December 2020  3
This article is made available to you with compliments of Apple

In [5]:
#See page 6
print(f"Page 6 ",apple[7].page_content)

Page 6  40 specialist teams: silicon design, camera software, reliabil-
ity engineering, motion sensor hardware, video engineering, 
core motion, and camera sensor design, to name just a few. 
How on earth does Apple develop and ship products that 
require such coordination? The answer is collaborative 
debate. Because no function is responsible for a product or a 
service on its own, cross-functional collaboration is crucial.
When debates reach an impasse, as some inevitably do, 
higher-level managers weigh in as tiebreakers, including at 
times the CEO and the senior VPs. To do this at speed with 
sufficient attention to detail is challenging for even the best 
of leaders, making it all the more important that the company 
fill many senior positions from within the ranks of its VPs, 
who have experience in Apple’s way of operating.
However, given Apple’s size and scope, even the executive 
team can resolve only a limited number of stalemates. The 
many horizontal dependencies mean th

In [6]:
#No of pages
len(apple)

11

##Chunking the data

In [7]:
#Data chunking

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name ="cl100k_base",
    chunk_size=512,
    chunk_overlap=20
)

In [8]:
document_chunks= pdf_loader.load_and_split(text_splitter)

In [9]:
len(document_chunks)

25

In [10]:
document_chunks[0].page_content

'REPRINT R2006F\nPUBLISHED IN HBR\nNOVEMBER–DECEMBER 2020\nARTICLE\nORGANIZATIONAL CULTURE\nHow Apple Is \nOrganized  \nfor Innovation\nIt’s about experts leading experts. \nby Joel M. Podolny and Morten T. Hansen\nThis article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.'

In [11]:
document_chunks[1].page_content

'2\nHarvard Business Review\nNovember–December 2020\nThis article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.'

In [12]:
document_chunks[-1].page_content

'to cultivate the experts-leading-experts model even within \na business unit structure. For example, when filling the next \nsenior management role, pick someone with deep expertise \nin that area as opposed to someone who might make the best \ngeneral manager. But a full-fledged transformation requires \nthat leaders also transition to a functional organization. \nApple’s track rec\xadord proves that the rewards may justify the \nrisks. Its approach can produce extraordinary results.\u2002\nHBR Reprint R2006F\nJOEL M. PODOLNY is a vice president of Apple and the dean  \nof Apple University. Prior to joining Apple, in 2009, he was  \nthe dean of the Yale School of Management and on the faculty of \nHarvard’s and Stanford’s business schools. MORTEN T. HANSEN  \nis a member of Apple University’s faculty and a professor at the \nUniversity of California, Berkeley. He was formerly on the faculties  \nof Harvard Business School and INSEAD.\nFOR ARTICLE REPRINTS CALL 800-988-0886 OR 617-783

##As expected there are some overlaps

This article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted

#If we increase the chunk length then overlap increases

## Embeddings

In [13]:
embedding_model = SentenceTransformerEmbeddings(model_name="thenlper/gte-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [16]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

In [17]:
print("Dimensions of the embedding vector", len(embedding_1))
print("Dimensions of the embedding vector", len(embedding_2))

print("Embedding vector",embedding_1)
print("Embedding vector",embedding_2)

Dimensions of the embedding vector 1024
Dimensions of the embedding vector 1024
Embedding vector [-0.007790964096784592, -0.0288176741451025, -0.027877110987901688, -0.02246643416583538, 0.022144237533211708, -0.006249097175896168, 0.008647195994853973, 0.016467047855257988, 0.018209118396043777, 0.053493041545152664, 0.013901167549192905, -0.00016959481581579894, 0.007971166633069515, -0.028715034946799278, 0.01758621260523796, -0.00955386646091938, -0.011853267438709736, -0.010937211103737354, 0.010688173584640026, 0.006078250240534544, 0.01815146952867508, 0.008558928966522217, -0.059489812701940536, -0.023468146100640297, -0.05630359798669815, 0.06647870689630508, 0.026126118376851082, 0.010339600034058094, 0.09469927102327347, 0.060104191303253174, -0.030409671366214752, -0.04133659973740578, 0.018987996503710747, -0.0794404149055481, -0.017287464812397957, -0.05677452310919762, 0.04051440581679344, -0.03622545301914215, -0.04527204483747482, -0.028730036690831184, -0.000266915187

Dimensions of embedding vector is 1024
All vector dimensions are same



*   The embedding vector provides a fixed length vector


## **Vector Database**

In [23]:
# Uninstall existing chromadb to avoid conflicts
!pip uninstall chromadb -y
# Install a specific, known-working version of chromadb
!pip install chromadb==0.4.18 -q

Found existing installation: chromadb 0.4.22
Uninstalling chromadb-0.4.22:
  Successfully uninstalled chromadb-0.4.22
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 9.8 MB/s eta 0:00:00


In [30]:
!pip show chromadb

Name: chromadb
Version: 0.4.18
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: bcrypt, chroma-hnswlib, fastapi, grpcio, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, overrides, posthog, pulsar-client, pydantic, pypika, PyYAML, requests, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 


In [41]:
import chromadb.settings
chromadb.settings.Settings(allow_reset=True, anonymized_telemetry=False)

ModuleNotFoundError: No module named 'chromadb.settings'

In [37]:
out_dir = 'apple_db'

In [38]:
if not os.path.exists(out_dir):
  os.makedirs(out_dir)


In [39]:
vectorstore = Chroma.from_documents(document_chunks,
                                    embedding_model,
                                    persist_directory=out_dir)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [40]:
vectorstore = Chroma(persist_directory=out_dir,embedding_function=embedding_model)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [42]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import os

out_dir = 'apple_db'
embedding_model = SentenceTransformerEmbeddings(model_name="thenlper/gte-large")

# Try creating the vectorstore without specifying persist_directory initially
vectorstore = Chroma.from_documents(document_chunks, embedding_model)

# Attempt to persist the vectorstore
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
vectorstore.persist()

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
